In [ ]:
#import Pkg; Pkg.add("JLD");Pkg.add("LinearAlgebra"), Pkg.add("DelimitedFiles"), Pkg.add("IterativeSolvers"), Pkg.add("Roots"), Pkg.add("SplitApplyCombine")
using JLD,LinearAlgebra, DelimitedFiles, Roots, Statistics, Distributions, IterativeSolvers, LinearMaps, Random, SplitApplyCombine
#SIMULATED DATA 
using JLD

In [ ]:
function gen_cov_matrix(m,vrank,ro,psi)    
    bsize = m*vrank
    covmatrix = zeros(bsize,bsize)
    for i in 1:bsize
        for j in 1:bsize
           if i == j 
               covmatrix[i,j] = 1
           elseif ceil(i/vrank) == ceil(j/vrank)
               covmatrix[i,j] = ro
           else
               covmatrix[i,j] = psi
           end
        end
    end
    return covmatrix 
end

function gen_cov_matrix2(m,vrank,ro,psi,gamma)    
    bsize = m*vrank
    covmatrix = zeros(bsize,bsize)
    for i in 1:bsize
        for j in 1:bsize
           if i == j 
               covmatrix[i,j] = 1
           elseif ceil(i/vrank) == ceil(j/vrank)
               covmatrix[i,j] = ro
           else
               covmatrix[i,j] = psi^(gamma*(abs(i-j)))
           end
        end
    end
    return covmatrix 
end

function simulate_data_gl(n,vrank,m,SNR,sigma)
    
    #X = hcat(zeros(n), gender)
    Z = Array{Matrix{Float64}}(undef, m)
    mean = zeros(vrank*m)
    mvn = MvNormal(mean, sigma)
    Zall = rand(mvn, n)'
    #Zall = (rand(n,vrank*m).<0.5).*Zall
    betasall = zeros(1,vrank*(m))
    # two variance components: additive and environment
    V2 = Array{Matrix{Float64}}(undef, m)
    betas = Array{Matrix{Float64}}(undef, m)
    indic =  Array{Integer}(undef, m)
    for i = 1:m
         #r = randn(n,vrank)
         #r =  r/(norm(r))^(1/4)
         #Z[i-1] = r
         #mat = r*r'
         #V2[i] =  mat
         #indic[i] = Int(rand()<0.01)
         indic[i] = Int(rand()<0.01)
         #indic[i]  =1
         #betas[i] = indic[i]*randn(vrank,1)*0.25
         betas[i] = indic[i]*randn(vrank,1)
         betasall[(i-1)*vrank+1:i*vrank] = betas[i]
         Z[i] =  Zall[:,(i-1)*vrank+1:i*vrank]
        
         #V2[i] =  r*r'
#         V2[i] =  r*r'
#         scale = norm(V2[i],2)
#         V2[i] =  V2[i]/scale#
    end
    print(sum(indic))
    print("\n")
    Zall = reduce(hcat, Z)
    #V = V2
    β = randn(m,1) # coef of sex effect
    σ2 = (randn(m,1).+1).^2  # variance of kinsip(additive), delta7(dominance),
        # H(householder) and error(environment)

    y = zeros(n,1)    
    for i = 1:m
        y += Z[i]*betas[i]
    end
    
    noise = betasall*(sigma*(betasall'))/SNR
    print(noise)
    y = y + noise[1]*randn(n)
    y = vec(y)
    σ2 = vec(σ2)
    return Z, Zall,betas ,σ2,y,betasall
end

In [26]:
n = 2000
vrank = 4
m = 5000

5000

In [27]:
#Within
ro = 0.5
#Between-group correlation
psi = 0.5

0.5

In [25]:
sigma = gen_cov_matrix(m,vrank,ro,psi);

InterruptException: InterruptException:

In [28]:
pall = m*vrank
sigma_inv = (1-ro)*I(pall)+ro*ones(pall)*ones(pall)'
#sigma = ((1-ro)^-1)*I(pall)-(ones(pall,pall)*ro)/((1-ro)^2+(1-ro)*pall*ro);
sigma = sigma_inv

20000×20000 Matrix{Float64}:
 1.0  0.5  0.5  0.5  0.5  0.5  0.5  0.5  …  0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  1.0  0.5  0.5  0.5  0.5  0.5  0.5     0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  1.0  0.5  0.5  0.5  0.5  0.5     0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  1.0  0.5  0.5  0.5  0.5     0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  1.0  0.5  0.5  0.5     0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  0.5  1.0  0.5  0.5  …  0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  0.5  0.5  1.0  0.5     0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  0.5  0.5  0.5  1.0     0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5     0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5     0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  …  0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5     0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  0.5  0.5  0.5 

In [ ]:
cond(sigma)

In [ ]:
maximum(eigvals(sigma))

In [29]:
Random.seed!(78)
#SIMULATE DATE
Z, Zall,betas ,σ2,y,betasall = simulate_data_gl(n,vrank,m,100,sigma)
#Z, Zall,betas ,σ2,y,betasall = simulate_data_glqr(n,vrank,m,1)
y = vec(y-mean(y)*ones(n,1));


#ORTHOGONALIZE MATRICES
Zq = Array{Matrix{Float64}}(undef, m)
Zqall = zeros(n,vrank*(m))
Rs = Array{Matrix{Float64}}(undef, m)
    for i = 1:m
         QRd = qr(Z[i])
         r = QRd.Q[:,1:vrank]
         Zq[i] = r
         Rs[i] = QRd.R
    end
Zqall = reduce(hcat, Zq);


gamma = ones(m)*(1/n);
#for j in 1:m
#    gamma[j] = maximum(eigen(Zq[j]*Zq[j]'/n).values)
#end

43
[2.3467817927226946;;]

In [30]:
maximum([norm(Zq[j]'*y) for j = 1:m])/2

261.6911663708152

In [31]:
maxl = round(maximum([norm(Zq[j]'*y) for j = 1:m])/2,digits = 2)

261.69

In [32]:
#maxl =  0.702377539

In [33]:
ngrid = 100

100

In [34]:
minl = maxl*1e-04

0.026169

In [35]:
minl = maxl*0.01

2.6169000000000002

In [36]:
lamrange = exp.(range(log(minl),log(maxl),length = ngrid))

100-element Vector{Float64}:
   2.6169000000000002
   2.7415056634752744
   2.8720445194187785
   3.0087990812563348
   3.1520653145032074
   3.3021532772954694
   3.459387791420751
   3.6241091453006264
   3.796673830446025
   3.977455312979533
   4.166844841894305
   4.365252295798834
   4.573107069980108
   ⋮
 156.87903346596357
 164.3489467413605
 172.17454556063893
 180.37276615872727
 188.96135120212912
 197.95888818777078
 207.38484967023663
 217.2596354044563
 227.60461649504765
 238.44218164786238
 249.7957856238352
 261.6899999999999

In [37]:
lamrange = reverse(lamrange)

100-element Vector{Float64}:
 261.6899999999999
 249.7957856238352
 238.44218164786238
 227.60461649504765
 217.2596354044563
 207.38484967023663
 197.95888818777078
 188.96135120212912
 180.37276615872727
 172.17454556063893
 164.3489467413605
 156.87903346596357
 149.74863927753583
   ⋮
   4.365252295798834
   4.166844841894305
   3.977455312979533
   3.796673830446025
   3.6241091453006264
   3.459387791420751
   3.3021532772954694
   3.1520653145032074
   3.0087990812563348
   2.8720445194187785
   2.7415056634752744
   2.6169000000000002

In [ ]:
size(lamrange)[1]

In [ ]:
lamdao = 2.9/n
lambda = lamdao*n

In [ ]:
#Tibsh Method P.Breheny Efficient
funtolall = 1e-20
#lambda1 = 10^3
@time begin
    hbeta2qstd,niter = group_lasso_std2_test_efficient(y,Zq,verbose = true, maxiter = 50000, vrank = vrank,gamma = gamma,funtol = funtolall, stop_comp = false,lambda = lamrange[100]);
end
print(niter)


In [38]:
w = ones(m);
w = w/sum(w);
#Experiment
funtolall = 1e-15
@time begin
   σ2auxw8, hbeta2qaux8,niter = group_lasso_aux10(y,Zq,Zqall,verbose = false , maxiter = 20*10^3,vrank = vrank,gamma = gamma,funtol = funtolall, stop_comp = false,lambda = lamrange[100], w = w);
end
print(niter)

 10.878269 seconds (2.67 M allocations: 8.602 GiB, 9.29% gc time, 2.44% compilation time)
822

In [ ]:
lamrange

In [ ]:
 sum(σ2auxw8.>0)

In [40]:
#Tibsh Method P.Breheny Efficient GRID
funtolall = 1e-15
#lambda1 = 10^3
@time begin
    iters_tibs,actives_tibs,beta_tibs,times_tibs = group_lasso_std2_test_efficient_grid(y,Zq,verbose = false, maxiter = 50000, vrank = vrank,gamma = gamma,funtol = funtolall, stop_comp = false,lambdas = lamrange);
end
#print(niter)

Index: 1
LAMBDA: 261.6899999999999
Iterations: 2
Active: 1.0
Loss: -------------------------
Index: 2
LAMBDA: 249.7957856238352
Iterations: 28
Active: 2.0
Loss: -------------------------
Index: 3
LAMBDA: 238.44218164786238
Iterations: 36
Active: 3.0
Loss: -------------------------
Index: 4
LAMBDA: 227.60461649504765
Iterations: 49
Active: 4.0
Loss: -------------------------
Index: 5
LAMBDA: 217.2596354044563
Iterations: 63
Active: 5.0
Loss: -------------------------
Index: 6
LAMBDA: 207.38484967023663
Iterations: 77
Active: 6.0
Loss: -------------------------
Index: 7
LAMBDA: 197.95888818777078
Iterations: 78
Active: 6.0
Loss: -------------------------
Index: 8
LAMBDA: 188.96135120212912
Iterations: 92
Active: 7.0
Loss: -------------------------
Index: 9
LAMBDA: 180.37276615872727
Iterations: 91
Active: 7.0
Loss: -------------------------
Index: 10
LAMBDA: 172.17454556063893
Iterations: 105
Active: 8.0
Loss: -------------------------
Index: 11
LAMBDA: 164.3489467413605
Iterations: 108


Iterations: 778
Active: 135.0
Loss: -------------------------
Index: 87
LAMBDA: 4.79085900570495
Iterations: 911
Active: 154.0
Loss: -------------------------
Index: 88
LAMBDA: 4.573107069980108
Iterations: 991
Active: 176.0
Loss: -------------------------
Index: 89
LAMBDA: 4.365252295798834
Iterations: 1124
Active: 205.0
Loss: -------------------------
Index: 90
LAMBDA: 4.166844841894305
Iterations: 1098
Active: 223.0
Loss: -------------------------
Index: 91
LAMBDA: 3.977455312979533
Iterations: 1080
Active: 243.0
Loss: -------------------------
Index: 92
LAMBDA: 3.796673830446025
Iterations: 1223
Active: 259.0
Loss: -------------------------
Index: 93
LAMBDA: 3.6241091453006264
Iterations: 1040
Active: 301.0
Loss: -------------------------
Index: 94
LAMBDA: 3.459387791420751
Iterations: 1144
Active: 325.0
Loss: -------------------------
Index: 95
LAMBDA: 3.3021532772954694
Iterations: 1416
Active: 354.0
Loss: -------------------------
Index: 96
LAMBDA: 3.1520653145032074
Iterations:

([2.0; 28.0; … ; 1197.0; 1120.0;;], [1.0; 2.0; … ; 470.0; 499.0;;], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0388208, 0.3159507, 0.4078469, 0.5733937, 0.7598188, 0.9287291, 0.9378708, 1.1161734, 1.1259328, 1.2454434  …  17.234013, 19.8117506, 16.9799484, 19.3071298, 24.1858862, 26.9731142, 30.0711367, 27.8483943, 22.9726183, 22.2528721])

In [ ]:
iters

In [39]:
#Aux Method GRID

w = ones(m);
w = w/sum(w);
#Experiment
funtolall = 1e-15
@time begin
   iters,actives,beta_aux,times_aux = group_lasso_aux10_grid(y,Zq,Zqall,verbose = false, maxiter = 20*10^3,vrank = vrank,gamma = gamma,funtol = funtolall, stop_comp = false,lambdas = lamrange, w = w);
end
#print(niter)

Index: 1
LAMBDA: 261.6899999999999
Iterations: 2
Active: 1.0
-------------------------
Index: 2
LAMBDA: 249.7957856238352
Iterations: 52
Active: 2.0
-------------------------
Index: 3
LAMBDA: 238.44218164786238
Iterations: 64
Active: 3.0
-------------------------
Index: 4
LAMBDA: 227.60461649504765
Iterations: 76
Active: 4.0
-------------------------
Index: 5
LAMBDA: 217.2596354044563
Iterations: 74
Active: 5.0
-------------------------
Index: 6
LAMBDA: 207.38484967023663
Iterations: 85
Active: 6.0
-------------------------
Index: 7
LAMBDA: 197.95888818777078
Iterations: 76
Active: 6.0
-------------------------
Index: 8
LAMBDA: 188.96135120212912
Iterations: 106
Active: 7.0
-------------------------
Index: 9
LAMBDA: 180.37276615872727
Iterations: 104
Active: 7.0
-------------------------
Index: 10
LAMBDA: 172.17454556063893
Iterations: 110
Active: 8.0
-------------------------
Index: 11
LAMBDA: 164.3489467413605
Iterations: 100
Active: 8.0
-------------------------
Index: 12
LAMBDA: 15

LAMBDA: 3.796673830446025
Iterations: 199
Active: 259.0
-------------------------
Index: 93
LAMBDA: 3.6241091453006264
Iterations: 252
Active: 301.0
-------------------------
Index: 94
LAMBDA: 3.459387791420751
Iterations: 256
Active: 325.0
-------------------------
Index: 95
LAMBDA: 3.3021532772954694
Iterations: 272
Active: 354.0
-------------------------
Index: 96
LAMBDA: 3.1520653145032074
Iterations: 262
Active: 375.0
-------------------------
Index: 97
LAMBDA: 3.0087990812563348
Iterations: 294
Active: 408.0
-------------------------
Index: 98
LAMBDA: 2.8720445194187785
Iterations: 321
Active: 439.0
-------------------------
Index: 99
LAMBDA: 2.7415056634752744
Iterations: 350
Active: 470.0
-------------------------
Index: 100
LAMBDA: 2.6169000000000002
Iterations: 370
Active: 499.0
-------------------------
 88.418808 seconds (12.25 M allocations: 18.614 GiB, 2.98% gc time, 0.09% compilation time)


([2.0; 52.0; … ; 350.0; 370.0;;], [1.0; 2.0; … ; 470.0; 499.0;;], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0149316, 0.36219, 0.4421768, 0.6166457, 0.5303991, 0.6244756, 0.545023, 0.770139, 0.7556842, 0.8317167  …  1.8432941, 1.9571382, 2.4915404, 2.5834643, 2.9090032, 2.6943642, 3.102961, 3.9050604, 4.4377361, 4.3326185])

In [ ]:
using DelimitedFiles


In [ ]:
writedlm( "julia_plots5.csv",  [cumsum(times_aux) cumsum(times_tibs) cumsum(iters[:,1]) cumsum(iters_tibs[:,1]) lamrange test_error],",");


In [ ]:
test_error = zeros(ngrid)
for k in 1:ngrid
    print(k)
    print(": ")
    test_error[k] = norm(betasall'-group_beta(beta_aux[:,k],Rs,m,vrank))
    print(norm(betasall'-group_beta(beta_aux[:,k],Rs,m,vrank)))
    print("\n")
end

In [ ]:
for k in 1:ngrid
    print(k)
    print(": \n")
    print(loss(Zq,y,beta_aux[:,k], vrank,m,n,lamrange[k]))
    print("\n")
    #print(loss(Zq,y,beta_tibs[:,k], vrank,m,n,lamrange[k]))
    print("\n")
end

In [ ]:
Pkg.add("GR")
using GR

function plot_cumulative_times(cumulative_times)
    plot(cumulative_times)
    xlabel("Iteration")
    ylabel("Cumulative Time (seconds)")
    title("Cumulative Time per Loop Iteration")
end

# Assuming cumulative_times is already defined from your loop
plot_cumulative_times(cumulative_times)

In [ ]:
# Plot the first dataset
    xlabel("log(lambda)")
p = plot(log.(lamrange),log.(cumsum(times_aux)))




# Optionally, save the combined plot to a file
#savefig(p, "combined_cumulative_times_plot.png")

In [ ]:
p = plot(log.(lamrange),log.(cumsum(times_tibs)))

In [49]:
for i in 1:100
    print(loss(Zq,y,beta_tibs[:,i], vrank,m,n,lamrange[i]))
    print("\n")
end

119.67463224743656
119.51829272887403
119.07501599169176
118.3767893743911
117.45057094799112
116.32466759816778
115.02739589584093
113.58522776179407
112.0206761067968
110.35533439643102
108.6081529076342
106.79608055892896
104.93399787121737
103.0349357214208
101.11074363458015
99.17185017982361
97.22738076162352
95.28529126538835
93.3515884154783
91.43141311003113
89.52990843547738
87.65141953755759
85.799585815056
83.97753988743446
82.18687215973586
80.42846739265991
78.70408496107045
77.01507343340599
75.3618357585672
73.74493172570926
72.16460485666643
70.62100347786291
69.11277712143618
67.63896833683984
66.19935819660319
64.79325827744458
63.420092447007185
62.078373720016664
60.76732160165203
59.48636692370052
58.23490234583267
57.01202721620462
55.81700427859765
54.64849130253677
53.50294100459656
52.377506412822676
51.268586901911256
50.17544124651923
49.096396101980076
48.03004413099942
46.97615646975595
45.93458764165011
44.90409564515879
43.88386710594922
42.8722263658633

In [51]:
for i in 1:100
    print(loss(Zq,y,beta_aux[:,i], vrank,m,n,lamrange[i]))
    print("\n")
end

119.67463224743658
119.51829272887443
119.07501599169194
118.3767893743915
117.45057094799066
116.32466759816043
115.02739589583389
113.58522776178623
112.0206761067923
110.35533439641813
108.608152907626
106.79608055890773
104.93399787119904
103.03493572139624
101.11074363455488
99.17185017980849
97.22738076160172
95.28529126538581
93.3515884154611
91.43141311001004
89.52990843545375
87.65141953752405
85.79958581503078
83.97753988740288
82.18687215970922
80.42846739263501
78.70408496104025
77.0150734333745
75.36183575853597
73.74493172567146
72.16460485662748
70.62100347781958
69.11277712137613
67.6389683367863
66.19935819653375
64.7932582773988
63.420092446962464
62.078373719964176
60.76732160160658
59.48636692366313
58.23490234579426
57.01202721616702
55.81700427856116
54.64849130248978
53.50294100453617
52.37750641275498
51.26858690185764
50.175441246439675
49.096396101938836
48.03004413091937
46.97615646967436
45.93458764158422
44.90409564506668
43.88386710588784
42.87222636578552

In [53]:
for i in 1:100
    print(deviance(Zq,y,beta_aux[:,i], vrank,m,n,lamrange[i]))
    print("\n")
end

119.67402179227928
112.92634497121266
106.72339786538757
100.80478991335539
95.29731840322675
90.17561831511672
85.4972247244679
81.18530534549366
77.23008744055656
73.61175174044176
70.29274516694946
67.25479047896665
64.46124760697984
61.899726918993814
59.55233435264425
57.39990404441125
55.42503353771837
53.61120752578595
51.907215736473425
50.335951541020265
48.88560933784753
47.54274302844665
46.2996010888621
45.14771201571202
44.031669590354
42.98709810152091
42.010380801774126
41.08887646453169
40.209583838406836
39.37852282175183
38.58568190476676
37.82737000972234
37.05987476113946
36.324662050556555
35.6134858327326
34.922242592744695
34.242845231377395
33.5662968578312
32.90847743108203
32.26825510182825
31.642111296124174
31.024828709396253
30.42176134776632
29.78276976456866
29.11024901416236
28.36564283310225
27.600599387885076
26.839844766778327
26.037017987783564
25.24063669906244
24.45311734990323
23.66919483748089
22.863448428008585
22.055515681952482
21.206210752493

In [54]:
for i in 1:100
    print(deviance(Zq,y,beta_tibs[:,i], vrank,m,n,lamrange[i]))
    print("\n")
end

119.67402179227923
112.92634636201947
106.7233978861252
100.80480058810231
95.29732171699878
90.17559908843876
85.49722127540443
81.18532363876021
77.23010508749583
73.61176489579888
70.29275174530477
67.25478034624594
64.46123878597786
61.899730468794175
59.552321633136366
57.399891517077485
55.42503355002621
53.611220484154344
51.90721746084134
50.33596296107686
48.88560608065156
47.54275534410164
46.299602209383735
45.14771499460055
44.03167034411087
42.987094689227554
42.01038702639414
41.08887561827043
40.209573786011276
39.378511590386175
38.58567209974771
37.82735781675509
37.05986187101632
36.32465690561068
35.6134791442478
34.92223959369498
34.24283960507844
33.56628898255146
32.908471681336266
32.2682551250216
31.64210723015733
31.024822483371423
30.421759355178125
29.782764735364626
29.110241095514596
28.36563851692862
27.60059391778455
26.8398428729625
26.03701538243284
25.240636950603527
24.45311726967713
23.669190987587488
22.86344991034548
22.055515085203517
21.206209222

In [ ]:
#Tibsh
loss(Zq,y,hbeta2qstd, vrank,m,n,lambda)

In [ ]:
# MM fixed weights
loss(Zq,y,hbeta2qaux8, vrank,m,n,lambda)

In [52]:
function group_beta(theta,Rs,m,vrank)
    beta = zeros(vrank*m,1)
    for j = 1:m
        beta[(j-1)*vrank+1:j*vrank] = Rs[j]\theta[(j-1)*vrank+1:j*vrank] 
    end
    return beta
end

function loss(Z,y,hbeta, vrank,m,n, lambda)
   Zall = reduce(hcat, Z);
   return ((0.5*norm(y- Zall*hbeta)^2) + sqrt(vrank)*lambda*sum([norm(hbeta[(j-1)*vrank+1:j*vrank]) for j = 1:m]))/n
end


function deviance(Z,y,hbeta, vrank,m,n, lambda)
   Zall = reduce(hcat, Z);
   return ((0.5*norm(y- Zall*hbeta)^2))/n
end


function soft_thresh(x,l)
    cons = max(0, 1-(l/abs(x)))
    return x*cons
end



soft_thresh (generic function with 1 method)

In [16]:
function group_lasso_std2_test_efficient(
    y::AbstractVector{T}, 
    Z::Vector{<:AbstractMatrix{T}};
    verbose::Bool = false, 
    maxiter::Integer = 5000, 
    funtol::Number = 1e-8,
    stop_comp::Bool = true,
    out_comp::Number = 1,
    vrank::Integer = 5000,
    gamma::Vector{T} = zeros(T, length(Z)),
    hbeta::Vector{T} = zeros(T, length(Z)*vrank),
    lambda::Number = 1
    ) where T <: LinearAlgebra.BlasFloat
    n, = size(y)
    m   = length(Z)   # no. variance components
    # pre-allocate working arrays
    # initialize algorithm
    r = zeros(n,1)
    r = deepcopy(y)
    #verbose && println("iter=", 0)
    #verbose && println(" σ2=", σ2)
    niter = maxiter
    hbetaold = zeros(m*vrank)
    Zjr = zeros(vrank,1)
    thetaj = zeros(vrank,1)
    dd = zeros(vrank,1)
    diffv = zeros(vrank,1)
    for iter in 1:maxiter
        dif = 0
        # update variance components
        hbetaold = deepcopy(hbeta)
        verbose && print("Iter: ")
        verbose && print(iter)
        verbose && print(", Active: ")
        verbose && print(sum(abs.(hbeta[1:vrank:end]).>0))
        verbose && print("\n")
        #@time begin
            for j in 1:m
                #r = r + F[j]
                #print(r)
                #Zj = Z[j]
                Zjr = Z[j]'*r+hbeta[(j-1)*vrank+1:j*vrank]
                term = 1-((sqrt(vrank)*lambda)/norm(Zjr)) 
                if term <= 0 
                    thetaj = zeros(vrank,1)
                    if ~(iszero(hbeta[(j-1)*vrank+1:j*vrank]))
                        r = r-Z[j]*(-hbeta[(j-1)*vrank+1:j*vrank])
                    end
                else
                    thetaj = term*Zjr
                    #F[j] = Zj*thetaj
                    #print(thetaj')
                    #print(hbeta[(j-1)*vrank+1:j*vrank]')
                    diffv = thetaj-hbeta[(j-1)*vrank+1:j*vrank]
                    r = r-Z[j]*diffv
                end
                hbeta[(j-1)*vrank+1:j*vrank] = thetaj
                dd=hbeta[(j-1)*vrank+1:j*vrank]- hbetaold[(j-1)*vrank+1:j*vrank]
                dif=max(dif,gamma[j]^2*(dd'*dd))
            end
        #end
        #verbose && println("iter=", iter)
        #verbose && println(" σ2=", σ2)
        #if maximum(abs.(σ2old - σ2)./(σ2.+1)) < funtol 
        #    niter = iter
        #    break
        #end
        if dif < funtol
            niter=iter
            break
        end
    end
    # output
    return hbeta, niter
end

group_lasso_std2_test_efficient (generic function with 1 method)

In [17]:
function group_lasso_std2_test_efficient_grid(
    y::AbstractVector{T}, 
    Z::Vector{<:AbstractMatrix{T}};
    verbose::Bool = false, 
    maxiter::Integer = 5000, 
    funtol::Number = 1e-8,
    stop_comp::Bool = true,
    out_comp::Number = 1,
    vrank::Integer = 5000,
    gamma::Vector{T} = zeros(T, length(Z)),
    hbeta::Vector{T} = zeros(T, length(Z)*vrank),
    lambdas::AbstractVector{T}
    ) where T <: LinearAlgebra.BlasFloat
    n, = size(y)
    m   = length(Z)   # no. variance components
    # pre-allocate working arrays
    # initialize algorithm
    r = zeros(n,1)
    r = deepcopy(y)
    #verbose && println("iter=", 0)
    #verbose && println(" σ2=", σ2)
    niter = maxiter
    hbetaold = zeros(m*vrank)
    Zjr = zeros(vrank,1)
    thetaj = zeros(vrank,1)
    dd = zeros(vrank,1)
    diffv = zeros(vrank,1)
    lamrange = size(lambdas)[1]
    iterations = zeros(lamrange,1)
    actives = zeros(lamrange,1)
    betall = zeros(m*vrank,lamrange)
    elapsed_times = Float64[]
    for lami in 1:lamrange
        lambda = lambdas[lami]
        print("Index: ")
        print(lami)
        print("\n")
        print("LAMBDA: ")
        print(lambda)
        print("\n")
        elapsed_time = @elapsed begin
            for iter in 1:maxiter
                dif = 0
                # update variance components
                hbetaold = deepcopy(hbeta)
                verbose && print("Iter: ")
                verbose && print(iter)
                verbose && print(", Active: ")
                verbose && print(sum(abs.(hbeta[1:vrank:end]).>0))
                verbose && print("\n")
                #@time begin
                    for j in 1:m
                        #r = r + F[j]
                        #print(r)
                        #Zj = Z[j]
                        Zjr = Z[j]'*r+hbeta[(j-1)*vrank+1:j*vrank]
                        term = 1-((sqrt(vrank)*lambda)/norm(Zjr)) 
                        if term <= 0 
                            thetaj = zeros(vrank,1)
                            if ~(iszero(hbeta[(j-1)*vrank+1:j*vrank]))
                                r = r-Z[j]*(-hbeta[(j-1)*vrank+1:j*vrank])
                            end
                        else
                            thetaj = term*Zjr
                            #F[j] = Zj*thetaj
                            #print(thetaj')
                            #print(hbeta[(j-1)*vrank+1:j*vrank]')
                            diffv = thetaj-hbeta[(j-1)*vrank+1:j*vrank]
                            r = r-Z[j]*diffv
                        end
                        hbeta[(j-1)*vrank+1:j*vrank] = thetaj
                        dd=hbeta[(j-1)*vrank+1:j*vrank]- hbetaold[(j-1)*vrank+1:j*vrank]
                        dif=max(dif,gamma[j]^2*(dd'*dd))
                    end
                #end
                #verbose && println("iter=", iter)
                #verbose && println(" σ2=", σ2)
                #if maximum(abs.(σ2old - σ2)./(σ2.+1)) < funtol 
                #    niter = iter
                #    break
                #end
                if dif < funtol
                    niter=iter
                    break
                end
                niter = maxiter
            end
        end
        push!(elapsed_times, elapsed_time)
        betall[:,lami] = hbeta
        iterations[lami] = niter
        actives[lami] = sum(abs.(hbeta[1:vrank:end]).>0)
        print("Iterations: ")
        print(niter)
        print("\n")
        print("Active: ")
        print(actives[lami])
        print("\n")
        print("Loss: ")
        #print(loss(Zq,y,hbeta, vrank,m,n,lambda))
        #print("\n")
        print("-------------------------\n")
    end
    return iterations,actives,betall,elapsed_times
end

group_lasso_std2_test_efficient_grid (generic function with 1 method)

In [18]:
#Optimal weight orignal cg indexing
function group_lasso_aux10(
    y::AbstractVector{T}, 
    Z::Vector{<:AbstractMatrix{T}},
    Zall::AbstractMatrix{T};
    verbose::Bool = false, 
    maxiter::Integer = 5000, 
    funtol::Number = 1e-8,
    stop_comp::Bool = true,
    out_comp::Number = 1,
    σ2::Vector{T} = zeros(T, length(Z)),    
    vrank::Integer = 5000,
    gamma::Vector{T} = zeros(T, length(Z)),
    hbeta::Vector{T} = zeros(T, length(Z)*vrank),
    w::Vector{T} = ones(T, length(Z))/length(Z),
    lambda::Number = 1
    ) where T <: LinearAlgebra.BlasFloat
    n, = size(y)
    m   = length(Z)   # no. variance components
    # pre-allocate working arrays
    # initialize algorithm
    #print(w)
    r = deepcopy(y)
    #verbose && println("iter=", 0)
    #verbose && println(" σ2=", σ2)
    niter = maxiter
    Gy = deepcopy(y)
    M = zeros(T, n, n)
    #X_A = zeros(T, n, vrank*m)
    X_An = zeros(T, n, 0)
    #gamma_A = zeros(T, 1, vrank*m)
    gamma_An =  Float64[]
    active =   Int[]
    ZGy = zeros(vrank,1)
    #Zj = zeros(n,vrank)
    ZGys = zeros(m*vrank,1)
    A = zeros(vrank, m)
    beta = zeros(m,1)
    #Evaluate Rankings
    for iter in 1:maxiter
        #print(gamma_A)
        dif = 0
        #σ2old = deepcopy(σ2)
        # update variance components
        hbetaold = deepcopy(hbeta)
        #print(σ2)
        verbose && print("Iter: ")
        verbose && print(iter)
        verbose && print(", Active: ")
        verbose && print(size(active)[1])
        verbose && print("\n")
        ZGys = Zall'*Gy
        A = reshape(ZGys,(vrank,m))
        beta = sqrt.(sum(A.^2,dims=1))'
        alpha = sqrt(vrank)*lambda*ones(m)
        #WEIGHT ASSIGNMENT
        w = getWeights(alpha, beta, σ2,m);
        for j in 1:m
            if (w[j]+σ2[j]) == 0
                continue
            end
            σ2jnew = soft_thresh((w[j]+σ2[j])*beta[j],w[j]*alpha[j])/(alpha[j])
            #σ2jnew = σ2jnew*(σ2jnew>1e-15)
            if σ2jnew != σ2[j]
                if  σ2jnew == 0
                    jr = findfirst(x->x==j,active)
                    splice!(active,  jr)
                    X_An = X_An[:,setdiff(1:end, (jr-1)*vrank+1:jr*vrank)]
                    gamma_An = gamma_An[setdiff(1:end, (jr-1)*vrank+1:jr*vrank)]
                elseif !(j in active)
                    active = [active; j]
                    gamma_An = [gamma_An; σ2jnew*ones(vrank)]
                    X_An = [X_An Zqall[:,(j-1)*vrank+1:j*vrank]] 
                    #X_An = [X_An Z[j]] 
                else
                    jr = findfirst(x->x==j,active)
                    gamma_An[(jr-1)*vrank+1:jr*vrank] = σ2jnew*ones(vrank)    
                end

                σ2[j] = σ2jnew                    
                bk = σ2[j]*(ZGys[(j-1)*vrank+1:j*vrank])
                hbeta[(j-1)*vrank+1:j*vrank] = bk
                dd=hbeta[(j-1)*vrank+1:j*vrank]- hbetaold[(j-1)*vrank+1:j*vrank]
                dif=max(dif,gamma[j]^2*(dd'*dd))
            end
        end

        lm = v -> v + X_An*(gamma_An.*(X_An'*v))
        LM  = LinearMap{Float64}(lm, n, n)
        cg!(Gy,LM,y, maxiter = 4)
        if dif < funtol
            niter=iter
            #print((0.5*norm(Gy)^2+sum((alpha).^2).*σ2))/n)
            #print("\n")
            break
        end
    end
    # output
    return σ2,hbeta, niter
end


group_lasso_aux10 (generic function with 1 method)

In [19]:
#Optimal weight orignal cg indexing
function group_lasso_aux10_grid(
    y::AbstractVector{T}, 
    Z::Vector{<:AbstractMatrix{T}},
    Zall::AbstractMatrix{T};
    verbose::Bool = false, 
    maxiter::Integer = 5000, 
    funtol::Number = 1e-8,
    stop_comp::Bool = true,
    out_comp::Number = 1,
    σ2::Vector{T} = zeros(T, length(Z)),    
    vrank::Integer = 5000,
    gamma::Vector{T} = zeros(T, length(Z)),
    hbeta::Vector{T} = zeros(T, length(Z)*vrank),
    w::Vector{T} = ones(T, length(Z))/length(Z),
    lambdas::AbstractVector{T}
    ) where T <: LinearAlgebra.BlasFloat
    n, = size(y)
    m   = length(Z)   # no. variance components
    # pre-allocate working arrays
    # initialize algorithm
    #print(w)
    r = deepcopy(y)
    #verbose && println("iter=", 0)
    #verbose && println(" σ2=", σ2)
    niter = maxiter
    Gy = deepcopy(y)
    M = zeros(T, n, n)
    #X_A = zeros(T, n, vrank*m)
    X_An = zeros(T, n, 0)
    #gamma_A = zeros(T, 1, vrank*m)
    gamma_An =  Float64[]
    active =   Int[]
    ZGy = zeros(vrank,1)
    #Zj = zeros(n,vrank)
    ZGys = zeros(m*vrank,1)
    A = zeros(vrank, m)
    beta = zeros(m,1)
    #Evaluate Rankings
    lamrange = size(lambdas)[1]
    iterations = zeros(lamrange,1)
    actives = zeros(lamrange,1)
    betall = zeros(m*vrank,lamrange)
    elapsed_times = Float64[]
    for lami in 1:lamrange
        print("Index: ")
        print(lami)
        print("\n")
        lambda = lambdas[lami]
        print("LAMBDA: ")
        print(lambda)
        print("\n")
        elapsed_time = @elapsed begin
            for iter in 1:maxiter
                #print(gamma_A)
                dif = 0
                #σ2old = deepcopy(σ2)
                # update variance components
                hbetaold = deepcopy(hbeta)
                #print(σ2)
                verbose && print("Iter: ")
                verbose && print(iter)
                verbose && print(", Active: ")
                verbose && print(size(active)[1])
                verbose && print("\n")
                ZGys = Zall'*Gy
                A = reshape(ZGys,(vrank,m))
                beta = sqrt.(sum(A.^2,dims=1))'
                alpha = sqrt(vrank)*lambda*ones(m)
                #WEIGHT ASSIGNMENT
                w = getWeights(alpha, beta, σ2,m);
                for j in 1:m
                    if (w[j]+σ2[j]) == 0
                        continue
                    end
                    σ2jnew = soft_thresh((w[j]+σ2[j])*beta[j],w[j]*alpha[j])/(alpha[j])
                    #σ2jnew = σ2jnew*(σ2jnew>1e-15)
                    if σ2jnew != σ2[j]
                        if  σ2jnew == 0
                            jr = findfirst(x->x==j,active)
                            splice!(active,  jr)
                            X_An = X_An[:,setdiff(1:end, (jr-1)*vrank+1:jr*vrank)]
                            gamma_An = gamma_An[setdiff(1:end, (jr-1)*vrank+1:jr*vrank)]
                        elseif !(j in active)
                            active = [active; j]
                            gamma_An = [gamma_An; σ2jnew*ones(vrank)]
                            X_An = [X_An Zqall[:,(j-1)*vrank+1:j*vrank]] 
                            #X_An = [X_An Z[j]] 
                        else
                            jr = findfirst(x->x==j,active)
                            gamma_An[(jr-1)*vrank+1:jr*vrank] = σ2jnew*ones(vrank)    
                        end

                        σ2[j] = σ2jnew                    
                        bk = σ2[j]*(ZGys[(j-1)*vrank+1:j*vrank])
                        hbeta[(j-1)*vrank+1:j*vrank] = bk
                        dd=hbeta[(j-1)*vrank+1:j*vrank]- hbetaold[(j-1)*vrank+1:j*vrank]
                        dif=max(dif,gamma[j]^2*(dd'*dd))
                    end
                end

                lm = v -> v + X_An*(gamma_An.*(X_An'*v))
                LM  = LinearMap{Float64}(lm, n, n)
                cg!(Gy,LM,y, maxiter = 4)
                #cg!(Gy,LM,y)
                if dif < funtol
                    niter=iter
                    #print((0.5*norm(Gy)^2+sum((alpha).^2).*σ2))/n)
                    #print("\n")
                    break
                end
            end
        end
        push!(elapsed_times, elapsed_time)
        betall[:,lami] = hbeta
        iterations[lami] = niter
        actives[lami] = size(active)[1]
        print("Iterations: ")
        print(niter)
        print("\n")
        print("Active: ")
        print(actives[lami])
        print("\n")
        #print("Loss: ")
        #print(loss(Zq,y,hbeta, vrank,m,n,lambda))
        #print("\n")
        print("-------------------------\n")
    end
    return iterations,actives,betall,elapsed_times
end


group_lasso_aux10_grid (generic function with 1 method)

In [20]:
function  getWeights(alpha, beta, gammat,J)
    d = alpha - beta;
    dsqr = d.^2;
    gb = gammat.*beta;
    idx = .~((d.>= 0).*(gammat .== 0));
    d = d[idx];
    dsqr = dsqr[idx];
    gb = gb[idx];
    a = sum(idx);
    s = sortperm(dsqr, rev = true);
    w = zeros(a,1); 
   # print(dsqr[s[1]])
   # print("\n")
    for k = 1:a
        v = dsqr[s[k]];
        if k > 1
            w[s[1:k-1]] = (gb[s[1:k-1]])./sqrt(v);
        end
        w[s[k]] = 1-sum(w[s[1:k-1]]);
        c = (gb[s[k]])/d[s[k]];
        if (0 < w[s[k]]) & ((w[s[k]] < c) | (d[s[k]] <= 0))
            break
        end
        v = sum(gb[s[1:k]])^2;
        if (dsqr[s[k]] >= v) & ((v >= dsqr[s[min(J,k+1)]]) | (k == a))
            w[s[1:k]] = gb[s[1:k]]./sqrt(v);
            break
        end
    end

    wfinal = zeros(J,1);
    wfinal[idx] = w;
    return wfinal
end

getWeights (generic function with 1 method)

In [21]:
function group_beta(theta,Rs,m,vrank)
    beta = zeros(vrank*m,1)
    for j = 1:m
        beta[(j-1)*vrank+1:j*vrank] = Rs[j]\theta[(j-1)*vrank+1:j*vrank] 
    end
    return beta
end

function loss(Z,y,hbeta, vrank,m,n, lambda)
   Zall = reduce(hcat, Z);
   return ((0.5*norm(y- Zall*hbeta)^2) + sqrt(vrank)*lambda*sum([norm(hbeta[(j-1)*vrank+1:j*vrank]) for j = 1:m]))/n
end


loss (generic function with 1 method)